The Taxi Data for NYC is stored in Parquet files and this takes up quite a bit of space. For space saving and consistency, we will first use this file to create a usable .csv file from the Taxi data .parquet files. 

In [1]:
import glob
import pandas as pd
import numpy as np

First we will create a glob with all of the files for easy itteration. During testing, it was discovered that 2009, 2010, and 2011-2022 all have different column names. We will pull one file from each year-set to determine the column names to use in the rest of our code.

In [2]:
taxi_data = sorted(glob.glob("Taxi Data\*.parquet"))
# df = pd.read_parquet("Taxi Data\yellow_tripdata_2010-01.parquet")
# df.columns

Originally, we needed to create 3 DataFrames to start with due to these differences. One for 2009 data, one for 2010 data, and a third for 2011-2022. We can accomplish this by looping over the enitre folder and including a try except block in our loop to catch any files with different column names. We needed to do this 3 times to extract all the data we need. The project shifted to only needing 2011-2022 so one time through will accomplish what we need.

In [3]:
temp_list_11_22 = []
col_list_11_22 = ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance',
       'PULocationID', 'DOLocationID','RatecodeID','store_and_fwd_flag', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount']

for file in taxi_data:
    try:
        df = pd.read_parquet(file, columns=col_list_11_22).sample(n=100000)
    except Exception as e:
        print(f"{file} Does not include correct cols.")
    else:
        temp_list_11_22.append(df)

Taxi Data\yellow_tripdata_2009-01.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-02.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-03.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-04.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-05.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-06.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-07.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-08.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-09.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-10.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-11.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-12.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2010-01.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2010-02.parquet Does not 

Next, we need to combine all of the data from each file into an overal DataFrame.

In [4]:
overall_data = pd.concat(temp_list_11_22)

With one DataFrame with all the data we need, we can do some inital cleaning that will carry through the CSV format.

We will:
    -Rename the columns to more user friendly names
    -Create a new column for just the year of each transaction for quicker access

In [5]:
cols_rename = ['VendorID', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID', 'RateCodeID', 'store_and_fwd_flag', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']
overall_data.columns = cols_rename

In [6]:
overall_data['year'] = overall_data["pickup_datetime"].dt.year

In [7]:
overall_data['is_covid_rel'] = np.where(overall_data["year"] > 2019, True, False)

In [8]:
overall_data = overall_data[overall_data.year >= 2011]
overall_data = overall_data[overall_data.year <= 2022]
overall_data.sort_values(by='year')

,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,RateCodeID,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,year,is_covid_rel
5730337,1,2011-01-14 22:50:03,2011-01-14 23:02:03,3.0,2.10,97,97,1.0,N,2,7.7,0.5,0.5,0.00,0.0,0.0,8.70,2011,False
2842983,2,2011-09-07 12:57:00,2011-09-07 13:11:00,1.0,1.00,161,230,1.0,None,2,8.1,0.0,0.5,0.00,0.0,0.0,8.60,2011,False
12267530,1,2011-09-26 10:56:13,2011-09-26 11:17:36,1.0,3.20,141,170,1.0,N,2,13.7,0.0,0.5,0.00,0.0,0.0,14.20,2011,False
6179550,1,2011-09-14 04:10:12,2011-09-14 04:31:59,5.0,5.30,249,37,1.0,N,1,16.1,0.5,0.5,4.27,0.0,0.0,21.37,2011,False
11736228,1,2011-09-25 01:41:24,2011-09-25 01:59:24,1.0,4.60,263,114,1.0,N,2,14.5,0.5,0.5,0.00,0.0,0.0,15.50,2011,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3010516,2,2022-04-27 10:24:45,2022-04-27 10:50:02,1.0,3.00,161,211,1.0,N,1,16.5,0.0,0.5,4.95,0.0,0.3,24.75,2022,True
200779,2,2022-04-02 16:19:26,2022-04-02 16:25:40,1.0,1.09,237,237,1.0,N,1,6.5,0.0,0.5,1.50,0.0,0.3,11.30,2022,True
509448,2,2022-04-05 14:03:29,2022-04-05 14:09:38,1.0,0.68,263,236,1.0,N,1,5.5,0.0,0.5,1.76,0.0,0.3,10.56,2022,True
2095944,2,2022-04-19 12:18:15,2022-04-19 12:43:53,1.0,13.75,132,223,1.0,N,2,38.5,0.0,0.5,0.00,0.0,0.3,40.55,2022,True


In [9]:
overall_data['payment_type'] = overall_data['payment_type'].values.astype(str).astype(int)

In [10]:
#change all 1 values to credit card
overall_data['payment_type'] = overall_data['payment_type'].replace(1, 'credit_card')
#change all 2 values to cash
overall_data['payment_type'] = overall_data['payment_type'].replace(2, 'cash')
#change all 3 values to no charge
overall_data['payment_type'] = overall_data['payment_type'].replace(3, 'no_charge')
#change all 4 values to dispute
overall_data['payment_type'] = overall_data['payment_type'].replace(4, 'dispute')
#change all 5 values to unknown
overall_data['payment_type'] = overall_data['payment_type'].replace(5, 'unknown')
#change all 6 values to voided trip
overall_data['payment_type'] = overall_data['payment_type'].replace(6, 'voided_trip')

Now we export to the CSV to be used in our main analysis.

In [11]:
overall_data.to_csv("overall_data.csv", index=False)